[흐름 요약]
1. 사용자 보험사 / 종류 선택 -> 벡터DB 주소 반환
2. 사용자 입력문 -> DB로드 -> 검색 -> 프롬프트 생성 -> 모델 답변 반환


In [ ]:
# 함수 1: 보험사와 보험종류로 FAISS/문서 경로 반환
def get_db_path(ins_name, ins_type):
    
    # 사용자가 선택한 보험사와 보험상품명을 기반으로 FAISS 및 문서 경로를 반환한다.
    base_dir = "./faiss_store"
    folder = f"{ins_name}_{ins_type}"  # 예: "hana_car"
    
    faiss_path = f"{base_dir}/{folder}/faiss_index.bin"
    doc_path = f"{base_dir}/{folder}/documents.pkl"
    
    return faiss_path, doc_path


In [ ]:
import os
import pickle
import requests
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

load_dotenv()
hf_api_key = os.getenv("HF_API_KEY")

In [ ]:
# 함수 2 : 사용자 질문 입력 → LLM 응답 생성 함수
def run_llm_query(query_text, faiss_path, meta_path, model_id="HuggingFaceH4/zephyr-7b-alpha"):

    # 사용자 질문(query_text)을 벡터DB에서 검색하고, 
    # 유사 문서 기반 프롬프트를 구성해 Hugging Face LLM 응답을 생성한다.

    # 1. 임베딩 모델 로드
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # 2. 벡터 DB 및 메타데이터 로드
    vector_db = FAISS.load_local(faiss_path, embeddings=embedding_model)
    retriever = vector_db.as_retriever()

    with open(meta_path, "rb") as f:
        documents = pickle.load(f)

    # 3. 유사 문서 검색
    top_docs = retriever.get_relevant_documents(query_text)[:3]

    # 4. 프롬프트 구성
    context_text = "\n\n".join([doc.page_content for doc in top_docs])
    llm_prompt = f"""[문서 내용]
{context_text}

[질문]
{query_text}

[답변]
"""

    # 5. Hugging Face LLM 응답 생성
    url = f"https://api-inference.huggingface.co/models/{model_id}"
    headers = {"Authorization": f"Bearer {hf_api_key}"}
    payload = {"inputs": llm_prompt, "parameters": {"max_new_tokens": 300}}

    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        generated = response.json()[0]["generated_text"]
        llm_response = generated.replace(llm_prompt, "").strip()
        return llm_response
    else:
        return f"⚠️ 오류 발생: {response.status_code} - {response.text}"
